# Probability Estimation and Calibration

<div align="justify">

The predictions delivered by corresponding methods are at best "pseudo-probabilities" that are often not very accurate. 
Besides, there are many methods that deliver natural scores, 
intuitively expressing a degree of confidence 
(like the distance from the separating hyperplane in support vector machines), 
but which do not immediately qualify as probabilities either. 
The idea of *scaling* or *calibration methods* is to turn such scores into proper, 
well-calibrated probabilities, that is, 
to learn a mapping from scores to the unit interval that can be applied to the output of a predictor as a kind of post-processing step ({cite:t}`flac_cc17`). 
Examples of such methods include binning ({cite:t}`zadr_oc01`), 
isotonic regression ({cite:t}`zadr_tc02`), 
logistic scaling ({cite:t}`Pla00`) 
and improvements thereof ({cite:t}`kull_bc17`), 
as well as the use of Venn predictors ({cite:t}`joha_vp18`). 
Calibration is still a topic of ongoing research. 

</div>